# 振子1实验：

## 使用SINDY测试

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import pysindy as ps
from pysindy.feature_library import GeneralizedLibrary, PolynomialLibrary, FourierLibrary
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso

# 自定义特征函数
def x_cos_x(x):
    return x * np.cos(x)

def x_sin_x(x):
    return x * np.sin(x)

def exp_x(x):
    return np.exp(x)

# 修复 function_names 参数，确保其为字符串列表
custom_library = ps.CustomLibrary(
    library_functions=[x_cos_x, x_sin_x, exp_x],
    function_names=[lambda x: "x*cos(x)", lambda x: "x*sin(x)", lambda x: "exp(x)"]  # 使用 lambda 确保正确的字符串表示
)

# 1. 读取数据
df = pd.read_csv('./test_ood.csv')
x = df['x'].values
v = df['v'].values
a = df['a'].values

X = np.vstack((x, v)).T
y = a.T

# 2. 构建并拟合 SINDy 模型
#    - 用多项式库（degree=5）来捕捉 sin(theta) 展开
#    - 用 STLSQ 稀疏回归
poly_lib = PolynomialLibrary(degree=3)            # [1, x, x^2, x^3]
fourier_lib = FourierLibrary(n_frequencies=2)      #  [1, sin(x), cos(x), sin(2x), cos(2x)]
library = GeneralizedLibrary(libraries=[poly_lib, fourier_lib, custom_library])
#optimizer = Lasso(alpha=0.001)  # 设置正则化强度
#optimizer = ps.STLSQ(threshold=0.1, alpha=0.01)  # 稀疏回归
optimizer = ps.SR3(threshold=0.1)  # 稀疏回归
model = ps.SINDy(feature_library=library, optimizer=optimizer)
model.fit(X, x_dot=a)
model.print()  # 打印识别出的方程

# 使用模型预测
predicted_values = model.predict(X)

# 计算均方误差 (MSE)
mse = mean_squared_error(a, predicted_values)
print(f"均方误差 (MSE): {mse}")



(x0)' = -0.500 x0 x1 + -0.200 x0^3 + -0.500 x1^3 + 0.800 sin(1 x0) + -1.000 x*cos(x)
均方误差 (MSE): 5.075484216875559e-32


c:\anacoda\envs\pytorch\Lib\site-packages\pysindy\optimizers\sr3.py:390: ConvergenceWarning: SR3._reduce did not converge after 30 iterations.
  warnings.warn(


# 使用PYSR测试

In [ ]:
import numpy as np
import pandas as pd
from pysr import PySRRegressor
import matplotlib.pyplot as plt

# 1. 读取数据
df = pd.read_csv('./train.csv')
x = df['x'].values
v = df['v'].values
a = df['a'].values

X = np.vstack((x, v)).T
y = a.T

# 定义 PySR 模型
model = PySRRegressor(
    niterations=100,  # 迭代次数
    populations=30,  # 种群数量
    binary_operators=["+", "-", "*", "/","^"],  # 二元操作符
    unary_operators=["sin", "cos","exp"],  # 一元操作符
    nested_constraints={
                "sin": {"cos": 0,"sin":0},  # 禁止 sin(cos(x))
                "exp": {"exp": 0,"cos": 0,"sin":0},  # 禁止 exp(exp(x))
                "cos": {"sin": 0,"cos": 0},
            },
    constraints={
        "^": (-1, 1)  # 再次尝试限制指数为常数
    },
    elementwise_loss="L2DistLoss()",  # 损失函数
    parallelism='multithreading',  # 启用多线程
    verbosity=1,  # 显示详细日志 (verbosity 而不是 verbose)
)

# 拟合模型
print("Fitting model...")
model.fit(X, y)

# 输出最佳表达式
for i, expr in enumerate(model.equations_):
    print(f"Best Expression for output {i+1}: {expr}")



Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython
Fitting model...
Compiling Julia backend...


c:\anacoda\envs\pytorch\Lib\site-packages\pysr\sr.py:2737: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
c:\anacoda\envs\pytorch\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


JuliaError: Operator log is not in the operator set.
Stacktrace:
 [1] error(s::String)
   @ Base .\error.jl:35
 [2] build_nested_constraints(; binary_operators::Any, unary_operators::Any, nested_constraints::Dict{Function, Dict{Function, Int64}})
   @ SymbolicRegression.CoreModule.OptionsModule C:\Users\19464\.julia\packages\SymbolicRegression\BInZG\src\Options.jl:113
 [3] Options(; defaults::Union{Nothing, VersionNumber}, binary_operators::Any, unary_operators::Any, maxsize::Union{Nothing, Integer}, maxdepth::Union{Nothing, Integer}, expression_type::Type{<:AbstractExpression}, expression_options::NamedTuple, node_type::Type{<:AbstractExpressionNode}, populations::Union{Nothing, Integer}, population_size::Union{Nothing, Integer}, ncycles_per_iteration::Union{Nothing, Integer}, elementwise_loss::Union{Nothing, Function, SupervisedLoss}, loss_function::Union{Nothing, Function}, dimensional_constraint_penalty::Union{Nothing, Real}, parsimony::Union{Nothing, Real}, constraints::Any, neste